# General information

![](https://miro.medium.com/max/884/1*UDi7KpyFX8gwV1k7aeMS-g.jpeg)

This was my first experience using a logistic regression for classification purpose, which I have done in the Data Science course from SkillFactory. The legend was that we have a dataset for credit scoring. The task was to build a model for predicting the probability of default of secondary clients. The development time was limited to 48 hours.

# 1. Initial setup

In [ ]:
# Importing modules.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

from pandas import Series
from sklearn import metrics 
from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor 
from sklearn.metrics import confusion_matrix, auc, roc_auc_score, roc_curve, accuracy_score, precision_score, recall_score, f1_score 

%matplotlib inline

In [ ]:
#Setting the conditions for experiments.
random_seed = 42
current_date = pd.to_datetime('21/10/2020')
pd.set_option('display.max_columns', None)
data_directory = '/kaggle/input/sf-dst-scoring/'
!pip freeze > requirements.txt

In [ ]:
# Defining a function for detecting outliers.
def outlier_detect(data, column):
    Q1 = np.percentile(column, 25)
    Q3 = np.percentile(column, 75)
    IQR = Q3 - Q1
    lower_range = Q1 - (1.5 * IQR)
    upper_range = Q3 + (1.5 * IQR)
    lower_number = len(data[column<lower_range])
    upper_number = len(data[column>upper_range])
    print('Lower Range:', lower_range,
          'Upper Range:', upper_range,
          'Lower Outliers:', lower_number,
          'Upper Outliers:', upper_number, 
          sep='\n')

In [ ]:
# Defining a function for visualization of confusion matrix.
def show_confusion_matrix(y_true, y_pred):
    color_text = plt.get_cmap('PuBu')(0.95)
    class_names = ['Default', 'Non-Default']
    cm = confusion_matrix(y_true, y_pred)
    cm[0,0], cm[1,1] = cm[1,1], cm[0,0]
    df = pd.DataFrame(cm, index=class_names, columns=class_names)
    
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.set(xticks=np.arange(cm.shape[1]), yticks=np.arange(cm.shape[0]), title="Confusion Matrix")
    ax.title.set_fontsize(15)
    sns.heatmap(df, square=True, annot=True, fmt="d", linewidths=1, cmap="PuBu")
    plt.setp(ax.get_yticklabels(), rotation=0, ha="right", rotation_mode="anchor", fontsize=12)
    plt.setp(ax.get_xticklabels(), rotation=0, ha="center", rotation_mode="anchor", fontsize=12)
    ax.set_ylabel('Predicted Values', fontsize=14, color = color_text)
    ax.set_xlabel('Real Values', fontsize=14, color = color_text)
    b, t = plt.ylim()
    plt.ylim(b+0.5, t-0.5)
    fig.tight_layout()
    plt.show()

In [ ]:
# Defining a function for visualization of metrics for logistic regression.
def all_metrics(y_true, y_pred, y_pred_prob):
    dict_metric = {}
    P = np.sum(y_true==1)
    N = np.sum(y_true==0)
    TP = np.sum((y_true==1)&(y_pred==1))
    TN = np.sum((y_true==0)&(y_pred==0))
    FP = np.sum((y_true==0)&(y_pred==1))
    FN = np.sum((y_true==1)&(y_pred==0))
    
    dict_metric['Positive, P'] = [P,'default']
    dict_metric['Negative, N'] = [N,'non-default']
    dict_metric['True Positive, TP'] = [TP,'correctly identified default']
    dict_metric['True Negative, TN'] = [TN,'correctly identified non-default']
    dict_metric['False Positive, FP'] = [FP,'incorrectly identified default']
    dict_metric['False Negative, FN'] = [FN,'incorrectly identified non-default']
    dict_metric['Accuracy'] = [accuracy_score(y_true, y_pred),'Accuracy=(TP+TN)/(P+N)']
    dict_metric['Precision'] = [precision_score(y_true, y_pred),'Precision = TP/(TP+FP)'] 
    dict_metric['Recall'] = [recall_score(y_true, y_pred),'Recall = TP/P']
    dict_metric['F1-score'] = [f1_score(y_true, y_pred),'Harmonical mean of Precision и Recall']
    dict_metric['ROC_AUC'] = [roc_auc_score(y_true, y_pred_prob),'ROC AUC Score']    

    temp_df = pd.DataFrame.from_dict(dict_metric, orient='index', columns=['Value', 'Description'])
    display(temp_df)   

In [ ]:
# Importing datasets.
data_train = pd.read_csv(data_directory+'train.csv')
data_test = pd.read_csv(data_directory+'test.csv')
sample_submission = pd.read_csv(data_directory+'/sample_submission.csv')

In [ ]:
# Checking the data.
data_train.info()
data_train.head()

In [ ]:
# Checking the data.
data_test.info()
data_test.head()

In [ ]:
# Merging the datasets.
data_train['sample'] = 1
data_test['sample'] = 0
data = data_train.append(data_test, sort=False).reset_index(drop=True)

# 2. Preliminary data examination & engineering


In [ ]:
# Checking the data.
data.info()
data.head()

In [ ]:
# Checking for missing values.
data.isna().sum()

In [ ]:
# Checking the number of unique values.
data.nunique()

* app_date - date of request, time variable, requires processing.
* education - level of education, categorical variable, requires processing and missing values correction.
* sex - binary variable, requires processing.
* age - continuous variable, requires processing.
* car - car availability, binary variable, requires processing.
* car_type - foreign-made car availability, binary variable, requires processing.
* decline_app_cnt - number of rejected requests, continuous variable.
* good_work - flag of a well-paid job, binary variable.
* score_bki - BKI (credit reporting agency) internal score, continuous variable.
* bki_request_cnt - number of requests to the BKI (credit reporting agency), continuous variable.
* region_rating - rating of the region, categorical variable.
* home_address - home address categorizer, categorical variable.
* work_address - work address categorizer, categorical variable.
* income - client's income level, continuous variable. 
* sna - level of connection with another clients, categorical variable.
* first_time - how long the client has been in the database, categorical variable.
* foreign_passport - passport availability, binary variable, requires processing.
* default - default in the past, binary target variable.  

In [ ]:
# Grouping column names by data type.
time_cols = ['app_date']
cat_cols = ['education', 'region_rating', 'home_address', 'work_address', 'sna', 'first_time']
bin_cols = ['sex', 'car', 'car_type', 'good_work', 'foreign_passport']
num_cols = ['age','decline_app_cnt','score_bki','bki_request_cnt','income']

In [ ]:
# Checking the missing values.
data['education'].value_counts(dropna = False)

Let's create a new binary variable for missing values in the "education"column.

In [ ]:
# Creating a new feature.
data['education_nan'] = pd.isna(data['education']).astype('uint8')

As a temporary measure, let's fill in the missing values with school education.

In [ ]:
# Filling the missing values with the most frequent value ('SCH').
data['education'] = data['education'].fillna('SCH')

Let's convert binary variables to numeric format using LabelEncoder.

In [ ]:
# Encoding binary variables.
label_encoder = LabelEncoder()
for column in bin_cols:
    data[column] = label_encoder.fit_transform(data[column])

In [ ]:
# Checking the data.
data.head()

# 3. Analysis of variables

## 3.1 Application date (+new feature: timedelta)

In [ ]:
# Converting the data to datetime.
data['app_date'] = pd.to_datetime(data['app_date'], format='%d%b%Y')
data.head()

Let's see what is the earliest request date in the dataset.

In [ ]:
# Finding the minimum.
data_min = min(data['app_date'])
data_min

Now let's introduce a new variable - the difference between the request date and the minimum.

In [ ]:
# Creating a new feature.
data['app_date_timedelta'] = (data['app_date'] - data_min).dt.days.astype('int')
data.head()

Let's add our new feature to the list of continuous variables.

In [ ]:
# Adding a feature to the list.
num_cols.append('app_date_timedelta')

In [ ]:
# Checking the frequency distribution.
data['app_date_timedelta'].hist(bins=50)

In [ ]:
# Checking the frequency distribution.
data.boxplot(column=['app_date_timedelta'])

In [ ]:
# Detection of outliers.
outlier_detect(data,data['app_date_timedelta'])

## 3.2 Education

In [ ]:
# Checking the frequency distribution.
data['education'].hist()

Everything looks logical: the higher the level of education, the fewer clients apply for a loan. Now let's convert education feature to numeric format using 'map' function.

In [ ]:
# Encoding a categorical variable.
education_dict = {
    'SCH': 1,
    'GRD': 2,
    'UGR': 3,
    'PGR': 4,
    'ACD': 5,
}

data['education'] = data['education'].map(education_dict)

## 3.3 Gender

In [ ]:
# Checking the frequency distribution.
data['sex'].hist(bins=2)

It seems that the bank has much more female clients.

## 3.4 Age

In [ ]:
# Checking the frequency distribution.
data['age'].hist()

In [ ]:
# Checking the frequency distribution.
data.boxplot(column=['age'])

In [ ]:
# Detection of outliers.
outlier_detect(data,data['age'])

People in their mid-30s are more likely to make big purchases, so it seems logical that we have a maximum there. Now let's log the data and look at the distribution.

In [ ]:
# Taking the logarithm.
np.log(data['age'] + 1).hist()

The distribution looks normal, so let's keep the logarithm.

In [ ]:
# Taking the logarithm.
data['age'] = np.log(data['age'] + 1)
data.head()

In [ ]:
# Checking the frequency distribution.
data.boxplot(column=['age'])

In [ ]:
# Detection of outliers.
outlier_detect(data,data['age'])

## 3.5 Car

In [ ]:
# Checking the frequency distribution.
data['car'].hist(bins=2)

Most of the clients in this dataset don't have a car.

## 3.6 Foreign-made car

In [ ]:
# Checking the frequency distribution.
data['car_type'].hist(bins=2)

Most car owners from this dataset have a car of domestic production.

## 3.7 Declined applications

In [ ]:
# Checking the frequency distribution.
data['decline_app_cnt'].hist()

In [ ]:
# Checking the frequency distribution.
data.boxplot(column=['decline_app_cnt'])

In [ ]:
# Detection of outliers.
outlier_detect(data,data['decline_app_cnt'])

The vast majority of borrowers do not have rejected applications, so our function treats a significant part of the results as outliers. Let's try to correct the situation by taking the logarithm of the column.

In [ ]:
# Taking the logarithm.
np.log(data['decline_app_cnt'] + 1).hist()

In [ ]:
# Taking the logarithm.
data['decline_app_cnt'] = np.log(data['decline_app_cnt'] + 1)

In [ ]:
# Checking the frequency distribution.
data.boxplot(column=['decline_app_cnt'])

In [ ]:
# Detection of outliers.
outlier_detect(data,data['decline_app_cnt'])

Unfortunately, the situation has not changed for the better.

## 3.8 Good work (well-paid job)

In [ ]:
# Checking the frequency distribution.
data['good_work'].hist(bins=2)

Most of the clients in this dataset don't have a well-paid job.

## 3.9 BKI score

In [ ]:
# Checking the frequency distribution.
data['score_bki'].hist()

In [ ]:
# Checking the frequency distribution.
data.boxplot(column=['score_bki'])

In [ ]:
# Detection of outliers.
outlier_detect(data,data['score_bki'])

The distribution looks normal, and there aren't many outliers.

## 3.10 BKI requests

In [ ]:
# Checking the frequency distribution.
data['bki_request_cnt'].hist()

In [ ]:
# Checking the frequency distribution.
data.boxplot(column=['bki_request_cnt'])

In [ ]:
# Detection of outliers.
outlier_detect(data,data['bki_request_cnt'])

In [ ]:
# Taking the logarithm.
np.log(data['bki_request_cnt'] + 1).hist()

In [ ]:
# Taking the logarithm.
data['bki_request_cnt'] = np.log(data['bki_request_cnt'] + 1)

In [ ]:
# Checking the frequency distribution.
data.boxplot(column=['bki_request_cnt'])

In [ ]:
# Detection of outliers.
outlier_detect(data,data['bki_request_cnt'])

Logarithmization significantly improved the situation, the number of outliers was reduced from 2636 to 15.


## 3.11 Region rating

In [ ]:
# Checking the frequency distribution.
data['region_rating'].hist()

## 3.12 Home address

In [ ]:
# Checking the frequency distribution.
data['home_address'].hist(bins=3)

## 3.13 Work address

In [ ]:
# Checking the frequency distribution.
data['work_address'].hist(bins=3)

## 3.14 Income

In [ ]:
# Checking the frequency distribution.
data['income'].hist(bins=100)

In [ ]:
# Checking the frequency distribution.
data.boxplot(column=['income'])

In [ ]:
# Checking the frequency distribution.
outlier_detect(data,data['income'])

In [ ]:
# Taking the logarithm.
np.log(data['income'] + 1).hist(bins=100)

In [ ]:
# Taking the logarithm.
data['income'] = np.log(data['income'] + 1)

In [ ]:
# Checking the frequency distribution.
data.boxplot(column=['income'])

In [ ]:
# Detection of outliers.
outlier_detect(data,data['income'])

Logarithmization improved the situation, the distribution changed from lognormal to normal, and the number of outliers decreased from 7000 to 2609.

## 3.15 SNA (level of connection with other clients)

In [ ]:
# Checking the frequency distribution.
data['sna'].hist()

## 3.16 First time (how long the client has been in the database)

In [ ]:
# Checking the frequency distribution.
data['first_time'].hist()

## 3.17 Foreign passport

In [ ]:
# Checking the frequency distribution.
data['foreign_passport'].hist(bins=2)

Most of the clients in this dataset don't have a foreign passport.

## 3.18 Default

In [ ]:
# Checking the frequency distribution.
data['default'].hist(bins=2)

The balance of classes in our dataset is strongly skewed towards customers who do not have a default.

# 4. Feature importance

In [ ]:
# Checking the correlation matrix
data_train_temp = data[data['sample']==1]
sns.heatmap(data_train_temp[num_cols].corr().abs(), vmin=0, vmax=1)

In [ ]:
# Checking the correlation matrix
data_train_temp[num_cols].corr().abs().sort_values(by='decline_app_cnt', ascending=False)

There were no strong correlations between the features, except for the number of declined applications and the BKI score. Unfortunately, I didn't have enough time to test the model with a different set of features.

In [ ]:
# Checking the frequency distribution.
fig, axes = plt.subplots(2, 3, figsize=(15, 15))
axes = axes.flatten()
for i in range(len(num_cols)):
    sns.boxplot(x="default", y=num_cols[i], data=data_train_temp, ax=axes[i])

Default clients are on average younger and earn less, have more applications and rejections from the bank, but simultaneously they have a higher BKI rating.

In [ ]:
# Checking the importance of features.
imp_num = Series(f_classif(data_train_temp[num_cols], 
                           data_train_temp['default'])[0], index = num_cols)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh')

In [ ]:
# Checking the importance of features.
imp_cat = Series(mutual_info_classif(
    data_train_temp[bin_cols + cat_cols], data_train_temp['default'], 
    discrete_features =True
), index = bin_cols + cat_cols)

imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh')

Of the continuous variables, the most important are the number of declined applications and the BKI rating. Of the categorical and binary non-variables, the most important are connections with other clients and the time spent in the database.

# 5. Data preprocessing¶


## 5.1 Standardization

In [ ]:
# Standardization of data.
ss = StandardScaler()
data[num_cols] = pd.DataFrame(ss.fit_transform(data[num_cols]),columns = data[num_cols].columns)

In [ ]:
# Checking the data.
data.info()
data.head(5)

## 5.2 Using RandomForestRegressor to fill missing values in "education" feature

Let's use the knowledge gained in [previous competition](https://www.kaggle.com/ogurrw/sf-tripadvisor-rating-akbar-murataliev) to predict the missing values in the "education"column.

In [ ]:
# Data processing and model training.
data_temp = data.drop(['sample', 'client_id', 'app_date', 'default'], axis=1)
data_education_nan = data_temp[data_temp['education_nan']==1]
data_no_nan = data_temp[data_temp['education_nan']==0]
y = data_no_nan['education'].values
X = data_no_nan.drop(['education'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = random_seed)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state = random_seed)
model.fit(X_train, y_train)
y_pred = np.round(model.predict(X_test))

In [ ]:
# Predicting the values.
predict = np.round(model.predict(data_education_nan.drop(['education'], axis=1)))

In [ ]:
# Adding predicted values to the dataset.
index_education_nan = data[data['education_nan']==1].index
data.loc[index_education_nan,'education'] = predict

## 5.3 One-hot encoding

In [ ]:
# Encoding categorical variables.
data = pd.get_dummies(data, prefix=cat_cols, columns=cat_cols)
data.info()

In [ ]:
# Checking the data.
data.head()

# 6. Model

In [ ]:
# Splitting the dataset.
data_train = data.query('sample == 1').drop(['sample', 'client_id', 'app_date'], axis=1)
data_test = data.query('sample == 0').drop(['sample', 'client_id', 'app_date'], axis=1)

In [ ]:
# Training and predicting.
X = data_train.drop(['default'], axis=1)
y = data_train['default'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=random_seed)

model = LogisticRegression()
model.fit(X_train, y_train)

y_pred_prob = model.predict_proba(X_test)[:,1]
y_pred = model.predict(X_test)

# 7. Model evaluation

In [ ]:
# Plotting the ROC curve
probs = model.predict_proba(X_test)
probs = probs[:,1]

fpr, tpr, threshold = roc_curve(y_test, probs)
roc_auc = roc_auc_score(y_test, probs)

plt.figure()
plt.plot([0, 1], label='Baseline', linestyle='--')
plt.plot(fpr, tpr, label = 'Regression')
plt.title('Logistic Regression ROC AUC = %0.3f' % roc_auc)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc = 'lower right')
plt.show()

In [ ]:
# Checking the confusion matrix.
show_confusion_matrix(y_test, y_pred)

In [ ]:
# Checking the metrics.
all_metrics(y_test, y_pred, y_pred_prob)

The ROC AUC score is good, but the confusion matrix shows us that our model predicts defaults very poorly. Of the 1827 defaults, only 40 were correctly predicted, or about 2 percent (very low Recall). Let's try to improve the situation by using custom hyperparameters.

# 8. Regularization

The code for regularization was taken from [here](https://www.kaggle.com/sokolovaleks/sf-dst-10-creditscoring-golobokov-sokolov) and slightly modified. Kudos to Alexandr Sokolov and Andrey Golobokov.

In [ ]:
model = LogisticRegression(random_state=random_seed)

iter_ = 50
epsilon_stop = 1e-3

param_grid = [
    {'penalty': ['l1'], 
     'solver': ['liblinear', 'lbfgs'], 
     'class_weight':['none', 'balanced'], 
     'multi_class': ['auto','ovr'], 
     'max_iter':[iter_],
     'tol':[epsilon_stop]},
    {'penalty': ['l2'], 
     'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'], 
     'class_weight':['none', 'balanced'], 
     'multi_class': ['auto','ovr'], 
     'max_iter':[iter_],
     'tol':[epsilon_stop]},
    {'penalty': ['none'], 
     'solver': ['newton-cg', 'lbfgs', 'sag', 'saga'], 
     'class_weight':['none', 'balanced'], 
     'multi_class': ['auto','ovr'], 
     'max_iter':[iter_],
     'tol':[epsilon_stop]},
]

gridsearch = GridSearchCV(model, param_grid, scoring='f1', n_jobs=-1, cv=5)
gridsearch.fit(X_train, y_train)
model = gridsearch.best_estimator_

best_parameters = model.get_params()
for param_name in sorted(best_parameters.keys()):
        print('\t%s: %r' % (param_name, best_parameters[param_name]))

preds = model.predict(X_test)
print('Accuracy: %.4f' % accuracy_score(y_test, preds))
print('Precision: %.4f' % precision_score(y_test, preds))
print('Recall: %.4f' % recall_score(y_test, preds))
print('F1: %.4f' % f1_score(y_test, preds))

We have identified optimal hyperparameters. Now let's train the model using these hyperparameters.

In [ ]:
# Training and predicting.
model = LogisticRegression(random_state=random_seed, 
                           C=1, 
                           class_weight='balanced', 
                           dual=False, 
                           fit_intercept=True, 
                           intercept_scaling=1, 
                           l1_ratio=None, 
                           multi_class='auto', 
                           n_jobs=None, 
                           penalty='l1', 
                           solver='liblinear', 
                           verbose=0, 
                           warm_start=False)

model.fit(X_train, y_train)

y_pred_prob = model.predict_proba(X_test)[:,1]
y_pred = model.predict(X_test)

In [ ]:
# Plotting the ROC curve
probs = model.predict_proba(X_test)
probs = probs[:,1]

fpr, tpr, threshold = roc_curve(y_test, probs)
roc_auc = roc_auc_score(y_test, probs)

plt.figure()
plt.plot([0, 1], label='Baseline', linestyle='--')
plt.plot(fpr, tpr, label = 'Regression')
plt.title('Logistic Regression ROC AUC = %0.3f' % roc_auc)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc = 'lower right')
plt.show()

In [ ]:
# Checking the confusion matrix.
show_confusion_matrix(y_test, y_pred)

In [ ]:
# Checking the metrics.
all_metrics(y_test, y_pred, y_pred_prob)

Thanks to the use of hyperparameters, Recall increased from 2 to 68 percent. Unfortunately, the Precision has decreased, because now our algorithm is more likely to detect defaults even where there are none. Harmonic mean between Recall and Precision (F1 Score) also decreased. However, from the point of view of the consumer, i.e. the bank, we can say that the quality of the model has increased. However, there is still enough room for improvement.

# 9. Submission

In [ ]:
data_train = data.query('sample == 1').drop(['sample', 'client_id', 'app_date'], axis=1)
data_test = data.query('sample == 0').drop(['sample', 'client_id', 'app_date'], axis=1)

In [ ]:
X_train = data_train.drop(['default'], axis=1)
y_train = data_train['default'].values
X_test = data_test.drop(['default'], axis=1)

In [ ]:
model = LogisticRegression(random_state=random_seed, 
                           C=1, 
                           class_weight='balanced', 
                           dual=False, 
                           fit_intercept=True, 
                           intercept_scaling=1, 
                           l1_ratio=None, 
                           multi_class='auto', 
                           n_jobs=None, 
                           penalty='l1', 
                           solver='liblinear', 
                           verbose=0, 
                           warm_start=False,
                           max_iter=1000)

model.fit(X_train, y_train)

In [ ]:
y_pred_prob = model.predict_proba(X_test)[:,1]

In [ ]:
submit = pd.DataFrame(data.query('sample == 0')['client_id'])
submit['default'] = y_pred_prob
submit.to_csv('submission.csv', index=False)

# 10. Recap & Conclusions

Let's follow the actions taken:

* We initialized necessary libraries, set visualization conditions and loaded the dataset.
* We analyzed the features, identified the target variable, looked at external sources, and suggested which features we can rely on for feature engineering.
* We checked each variable, frequency distributions and created several new features.
* We filled in the missing values by training the model on the available data.
* We encoded categorical variables and standardized the data.
* We trained logistic regression on the available data and evaluated the quality of its prediction using confusion matrix, ROC AUC, and other metrics.
* We selected hyperparameters and trained the model on them, improving its consumer qualities.

The following conclusions can be drawn from the results:
* We can fill in the missing values using a prediction model based on the available data.
* Built-in sklearn features make pre-processing of data much easier.
* We should not blindly focus on metrics, the consumer qualities of the model are also very important.